In [ ]:
#Instalação das libs
#!pip install yfinance
#!pip install crewai
#!pip install 'crewai[tools]'

#!python.exe -m pip install --upgrade pip


#python.exe -m pip install --upgrade pip

#!pip install duckduckgo_search

#!pip install langchain
#!pip install langchain-openai
#!pip install langchain-community


#!pip install crewai crewai-tools

In [88]:
#Import das libs
import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

In [89]:
#Criando Tool Fincance Tool
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-16")
    return stock

yahoo_finance_tool = Tool(
    name = "Yahoo finance Tool",
    description = "Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
    func= lambda ticket: fetch_stock_price(ticket)
)

In [90]:
#Importando OPENAI LLM - GPT
os.environ['OPENAI_API_KEY'] = "SecretKEY"
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [91]:
stockPriceAnalyst = Agent(
    role="Senior stock price Analyst",
    goal="Find the {ticket} stock price and analyses trends",
    backstory=""""You're a highly experienced in analyzing the price of an specific stock and make predictions about its future prices.""",
    verbose= True,
    llm= llm,
    max_iter= 5,
    memory= True,
    allow_delegation= False,
    tools=[yahoo_finance_tool]
)

In [92]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output="""" Specify the current trend stock price - up, down or sideways.
     eg. stock= 'APPL, price UP' """,
     agent= stockPriceAnalyst
)

In [77]:
#import a tool de search
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [93]:
newsAnalyst = Agent(
    role="Stock News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with the news context.
    For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory=""""You're a highly experienced in analyzing the marcket trends and news and have tracked assets for more then 10 years.

    You're also master level in analyst in the tradicional markets and have deep undestanding of human psychology.

    You undestand news, theirs tittles and information, but you look at those with a health dose of skepticism. You consider also the source of the news articles.
    """,
    verbose= True,
    llm= llm,
    max_iter= 10,
    memory= True,
    allow_delegation= False,
    tools=[search_tool]
)

In [94]:
get_news = Task(
    description=f"""Take the stock and always include BTC to it (if not request).
    Use the search tool to search each one individually.

    The current date is {datetime.now()},

    Compose the results into a helpfull report""",
    expected_output ="""A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent=newsAnalyst
)

In [95]:
stockAnalystWrite = Agent(
    role="Stock Senior Analyst Writer",
    goal="""Analyze the trends price and news and write an insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend.""",
    backstory=""""You're a widely accepted as the best stock analyst in the market. You undestand complex concepts and create compelling stories 
    and narratives that resonate with wider audiences.

    You undestand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. You are able to hold multiple opinions when analyzing anything.
    """,
    verbose= True,
    llm= llm,
    max_iter= 5,
    memory= True,
    allow_delegation= True
)

In [96]:
writeAnalyses = Task(
    description=""""Use the stock price trend and stock news report to create an analyses and write the newsletter about the {ticket} company
    tha is brief and highlights the most importante points.
    Focus on the stock price trend, news and fear/greed score. What are the newa future considerations?
    Include the previos analyses of stock trend and news summary 
    """,
    expected_output=""""An eloquent 3 paragraphs newsletter formated as markdown in an easy manner. It should contain:

    -3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fear/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways
    """,
    agent= stockAnalystWrite,
    context = [getStockPrice, get_news]
)

In [97]:
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
    tasks = [getStockPrice, get_news, writeAnalyses],
    verbose= True,
    process = Process.hierarchical,
    full_output = True,
    share_crew = False,
    manager_llm = llm,
    max_iter = 15
)

2024-08-20 18:42:10,665 - 15488 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [98]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

#response = yahoo_finance_tool.run("AAPL")

 [2024-08-20 18:42:13][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 18:42:13][INFO]: == Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-20 18:43:41][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 18:43:41][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 18:43:41][INFO]: == Starting Task: Take the stock and always include BTC to it (if not request).
    Use the search tool to search each one individually.

    The current date is 2024-08-20 18:42:04.338359,

    Compose the results into a helpfull report


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-20 18:45:10][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 18:45:10][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 18:45:10][INFO]: == Starting Task: "